<a href="https://colab.research.google.com/github/Candykick/2019_HUHS_Mentoring/blob/master/tensorflow_study_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lab 09-1: Neural Net for XOR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# random seed 설정
# 이는 매 실행 때마다 같은 값을 얻기 위함.
tf.random.set_seed(777)

# 원본 데이터셋
# XOR 문제를 풀기 위한 데이터이므로, 같은 값이면 0, 다른 값이면 1을 출력해야 한다.
x_data = [[0, 0],[0, 1],[1, 0],[1, 1]]
y_data = [[0],[1],[1],[0]]

# Tensorflow data API를 통해 학습시킬 값들을 담는다 (Batch Size는 한번에 학습시킬 Size로 정한다)
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

print(dataset)

<BatchDataset shapes: ((None, 2), (None, 1)), types: (tf.int32, tf.int32)>


In [ ]:
# preprocess function으로 features, labels는 실재 학습에 쓰일 Data 연산을 위해 Type를 맞춰준다.
# Int -> Flaot32로 Casting
def preprocess_data(features, labels):
  features = tf.cast(features, tf.float32)
  labels = tf.cast(labels, tf.float32)
  return features, labels

# Weight, bias 초기값 지정
W1 = tf.Variable(tf.random.normal([2, 1]), name='weight1')
b1 = tf.Variable(tf.random.normal([1]), name='bias1')

W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1]), name='bias2')

W3 = tf.Variable(tf.random.normal([2, 1]), name='weight3')
b3 = tf.Variable(tf.random.normal([1]), name='bias3')

# 3개의 Logistic Regression Unit을 합친 Neural Net 함수.
# Layer 1을 Vector로 합치지 않은 버전.
def neural_net(features):
  layer1 = tf.sigmoid(tf.matmul(features, W1)+b1)
  layer2 = tf.sigmoid(tf.matmul(features, W2)+b2)
  layer3 = tf.concat([layer1, layer2], -1)
  layer3 = tf.reshape(layer3, shape = [-1, 2])
  hypothesis = tf.sigmoid(tf.matmul(layer3, W3)+b3)
  return hypothesis

# Loss function
def loss_fn(hypothesis, labels):
  cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1-labels)*tf.math.log(1-hypothesis))
  return cost

# Gradient Descent Optimizert Setting
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Measure Accuracy Function
def accuracy_fn(hypothesis, labels):
  predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
  return accuracy

# Gradient Descent Function
def grad(features, labels):
  with tf.GradientTape() as tape:
    loss_value = loss_fn(neural_net(features), labels)
  return tape.gradient(loss_value, [W1, W2, W3, b1, b2, b3])

In [ ]:
# Repeat Count
EPOCHS = 50000

# Do Gradient Descent
for step in range(EPOCHS):
  for features, labels in dataset:
    features, labels = preprocess_data(features, labels)
    grads = grad(features, labels)
    optimizer.apply_gradients(grads_and_vars=zip(grads, [W1, W2, W3, b1, b2, b3]))
    if step % 5000 == 0:
      print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(neural_net(features), labels)))
    
# Measure Accuracy of the Neural_Net
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data), y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 0.8487
Iter: 5000, Loss: 0.6847
Iter: 10000, Loss: 0.6610
Iter: 15000, Loss: 0.6154
Iter: 20000, Loss: 0.5722
Iter: 25000, Loss: 0.5433
Iter: 30000, Loss: 0.5211
Iter: 35000, Loss: 0.4911
Iter: 40000, Loss: 0.4416
Iter: 45000, Loss: 0.3313
Testset Accuracy: 1.0000


### Lab 09-2: Tensorboard (Neural Net for XOR)

### Lab 10-1: Sigmoid 보다 ReLU가 더 좋아

어떤 프로젝트인가 : mnist 데이터셋을 이용해서 숫자 필기체 이미지를 받아온 뒤, 이를 Activation Function을 바꿔가면서 학습, 정확도를 비교한다.
<br><br>
참고 자료 : https://tensorflow.blog/케라스-딥러닝/2-시작하기-전에-신경망의-수학적-구성-요소/
<br><br>
※ Tensorboard 추가하는 것도 알아봤는데, 현 구조 자체가 콜백 함수를 적용하기에 적절하지 않아서(정확히는 어디다가 넣어야 할지를 몰라서) 추가하지 않았다.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist # fasion_mnist, cifar10, cifar100과 같은 다양한 데이터셋이 있다.

# mnist 데이터셋을 받아온 뒤 전처리.
def load_mnist():
  # train은 6만 장, test는 1만 장.
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

  # expand_dims : 채널 정보를 추가해서 input의 shape 변경.
  # tensorflow가 input으로 받는 shape는 [batch_size, height, width, channel]과 같은 형식이어야 한다.
  # numpy.expand_dims(array, axis) : Expand the shape of an array. 
  #                                  Insert a new axis that will appear at the axis position in the expanded array shape.
  #                                  axis로 지정된 차원을 추가한다. axis=-1인 경우, 마지막에 차원을 추가한다.
  train_data = np.expand_dims(train_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)
  test_data = np.expand_dims(test_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)

  # 이미지들의 숫자값을 0~1사이의 값으로 정규화.
  train_data, test_data = normalize(train_data, test_data) # 범위는 0~255 -> 0~1

  # labels 전처리
  # 10 : 데이터셋의 라벨 갯수(여기서는 0~9까지의 숫자 갯수이므로 10개.)
  # to_categorical(): One-Hot Incoding을 제공하는 함수. 여기서는 labels 값들을 One-hot Incoding된 값으로 변환해 준다.
  train_labels = to_categorical(train_labels, 10)
  test_labels = to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels

# 정규화 함수.
# 범위가 255까지이므로 255로 나눠 주면 된다.
# float32로 형변환 필수.
def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0

  return train_data, test_data

In [ ]:
# shape를 펼쳐주는 함수
# 다차원 배열을 1차원으로 만드는 레이어를 추가한다고 보면 된다.
def flatten():
  return tf.keras.layers.Flatten()

# Dense Layer(Flip Connected Layer) 사용, 케라스 레이어에 Dense 추가
def dense(channel, weight_init):
  # units = output으로 나가는 채널 갯수, use_bias = bias 사용 여부
  return tf.keras.layers.Dense(units=channel, use_bias=True, kernel_initializer=weight_init)

# Relu Activation Function
def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

# Sigmoid Activation Function
def sigmoid():
  return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

model에서 레이어 배치 순서는 다음과 같다.

가장 많이 사용하는 순서는<br>
layer<br>
normalization<br>
activation

혹은

normalization<br>
activation<br>
layer

In [ ]:
# Class type의 모델.
class create_model(tf.keras.Model): # tf.keras.Model을 상속받아야 한다.
  def __init__(self, label_dim): # label_dim : 몇 개의 output을 낼 것인지를 알려준다. 여기서는 10.
    super(create_model, self).__init__()

    weight_init = tf.keras.initializers.RandomNormal() # weight_init을 랜덤하게 생성. (평균이 0, 분산이 1인 가우시안 분포)
    self.model = tf.keras.Sequential() # 리스트 자료구조 타입으로, 네트워크 구조의 레이어들을 담고 있다고 보면 되나?

    # model에 flatten 레이어 추가. 이미지 shape를 펼쳐 준다.
    self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]

    for i in range(2):
      # [N, 784] -> [N, 256] -> [N, 256]
      self.model.add(dense(256, weight_init)) # flip-connected function * 2
      self.model.add(relu()) # relu function * 2

    # 네트워크의 logits를 구함. 10개의 output으로 출력.
    self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
 
 # 위에서 만든 model을 실제로 호출하는 함수.
  def call(self, x, training=None, mask=None):
    x = self.model(x)

    return x

In [ ]:
# loss 값 구하는 함수.
def loss_fn(model, images, labels):
  logits = model(images, training=True) # images의 숫자를 추출.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)) # softmax로 loss 구함.
  return loss

# 정확도 측정 함수.
# argmax : 가장 큰 숫자값의 위치를 찾는 함수. 여기서는 마지막(-1번째) 차원을 제거한다.
def accuracy_fn(model, images, labels):
  logits = model(images, training=False)
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) # prediction은 Boolean 이기 때문에, 이를 숫자값으로 변경.
  return accuracy

# Gradient Descent 함수.
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)

In [ ]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128 # 이미지를 한 번에 학습시킬 갯수.

training_epochs = 1
training_iterations = len(train_x)

label_dim = 10

""" Graph Input using Dataset API """
# 데이터셋 API를 이용해서 각각의 이미지/labels들을 네트워크에 넣는 것 구현.
# 한 번에 다 넣으면 메모리에 부담이 되므로, batch_size만큼만 넣는다.
# shuffle : 데이터셋을 잘 섞으라는 의미. 여기서 buffer_size는 input data보다 갯수가 많은 값이면 된다.
# prefetch : 네트워크가 batch_size만큼 학습중일 때, 미리 메모리에 다음 batch_size만큼 학습데이터를 올려둔다.
# batch : batch를 batch_size만큼 진행, 네트워크에 자료를 넣는다.
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=batch_size).\
  batch(batch_size, drop_remainder=True)
  #repeat() #이를 계속 반복.

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=len(test_x)).\
  batch(len(test_x))
  #repeat()

""" Model """
network = create_model(label_dim)

""" Training """
# 어떤 Optimizer를 써서 loss값을 최소화할 것인가 : 여기서는 AdamOptimizer를 사용.
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [ ]:
# 여기선 체크포인트 구현은 생략했다. (핵심도 아니고, 귀찮고, 코랩 쓰고 있다보니.......)
start_epoch = 0
start_iteration = 0

for epoch in range(start_epoch, training_epochs):
  for idx, (train_input, train_label) in enumerate(train_dataset): #epoch, iteration 2개에 대해 반복 수행.
    # gradient 구한 후 적용, 네트워크를 학습시킨다.
    grads = grad(network, train_input, train_label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

    # loss, 정확도를 구한다.
    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    # test dataset을 불러오고, 정확도를 구한다.
    for test_input, test_label in test_dataset:
      test_accuracy = accuracy_fn(network, test_input, test_label)

    # 그 결과값을 출력한다.
    print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" %(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))

Epoch: [ 0] [    0/60000], train_loss: 2.02356291, train_accuracy: 0.4297, test_Accuracy: 0.4792
Epoch: [ 0] [    1/60000], train_loss: 1.96166503, train_accuracy: 0.5391, test_Accuracy: 0.5927
Epoch: [ 0] [    2/60000], train_loss: 1.83400893, train_accuracy: 0.6484, test_Accuracy: 0.6899
Epoch: [ 0] [    3/60000], train_loss: 1.75676799, train_accuracy: 0.7188, test_Accuracy: 0.7443
Epoch: [ 0] [    4/60000], train_loss: 1.66362655, train_accuracy: 0.7422, test_Accuracy: 0.7710
Epoch: [ 0] [    5/60000], train_loss: 1.56959581, train_accuracy: 0.7734, test_Accuracy: 0.7648
Epoch: [ 0] [    6/60000], train_loss: 1.44071627, train_accuracy: 0.7891, test_Accuracy: 0.7513
Epoch: [ 0] [    7/60000], train_loss: 1.38239574, train_accuracy: 0.7188, test_Accuracy: 0.7619
Epoch: [ 0] [    8/60000], train_loss: 1.24083471, train_accuracy: 0.7969, test_Accuracy: 0.7776
Epoch: [ 0] [    9/60000], train_loss: 1.19959927, train_accuracy: 0.7109, test_Accuracy: 0.7843
Epoch: [ 0] [   10/60000], tra

<h1>Relu Function Test Accuracy : 96.25%</h1>

---



---



In [ ]:
# Class type의 모델 2번째 : Sigmoid를 사용한다.
class create_model_sigmoid(tf.keras.Model): # tf.keras.Model을 상속받아야 한다.
  def __init__(self, label_dim): # label_dim : 몇 개의 output을 낼 것인지를 알려준다. 여기서는 10.
    super(create_model_sigmoid, self).__init__()

    weight_init = tf.keras.initializers.RandomNormal() # weight_init을 랜덤하게 생성. (평균이 0, 분산이 1인 가우시안 분포)
    self.model = tf.keras.Sequential() # 리스트 자료구조 타입으로, 네트워크 구조의 레이어들을 담고 있다고 보면 되나?

    # model에 flatten 레이어 추가. 이미지 shape를 펼쳐 준다.
    self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]

    for i in range(2):
      # [N, 784] -> [N, 256] -> [N, 256]
      self.model.add(dense(256, weight_init)) # flip-connected function * 2
      self.model.add(sigmoid()) # relu function * 2

    # 네트워크의 logits를 구함. 10개의 output으로 출력.
    self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
 
 # 위에서 만든 model을 실제로 호출하는 함수.
  def call(self, x, training=None, mask=None):
    x = self.model(x)

    return x

In [ ]:
""" Model """
network = create_model_sigmoid(label_dim)

# 여기선 체크포인트 구현은 생략했다. (핵심도 아니고, 귀찮고, 코랩 쓰고 있다보니.......)
start_epoch = 0
start_iteration = 0

for epoch in range(start_epoch, training_epochs):
  for idx, (train_input, train_label) in enumerate(train_dataset): #epoch, iteration 2개에 대해 반복 수행.
    # gradient 구한 후 적용, 네트워크를 학습시킨다.
    grads = grad(network, train_input, train_label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

    # loss, 정확도를 구한다.
    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    # test dataset을 불러오고, 정확도를 구한다.
    for test_input, test_label in test_dataset:
      test_accuracy = accuracy_fn(network, test_input, test_label)

    # 그 결과값을 출력한다.
    print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" %(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))

Epoch: [ 0] [    0/60000], train_loss: 2.26130342, train_accuracy: 0.2422, test_Accuracy: 0.1680
Epoch: [ 0] [    1/60000], train_loss: 2.30395555, train_accuracy: 0.1406, test_Accuracy: 0.1071
Epoch: [ 0] [    2/60000], train_loss: 2.31546450, train_accuracy: 0.1094, test_Accuracy: 0.1010
Epoch: [ 0] [    3/60000], train_loss: 2.22051382, train_accuracy: 0.2578, test_Accuracy: 0.2212
Epoch: [ 0] [    4/60000], train_loss: 2.26138830, train_accuracy: 0.1328, test_Accuracy: 0.0982
Epoch: [ 0] [    5/60000], train_loss: 2.20962024, train_accuracy: 0.0938, test_Accuracy: 0.0982
Epoch: [ 0] [    6/60000], train_loss: 2.13300753, train_accuracy: 0.2031, test_Accuracy: 0.2253
Epoch: [ 0] [    7/60000], train_loss: 2.10313439, train_accuracy: 0.2891, test_Accuracy: 0.3122
Epoch: [ 0] [    8/60000], train_loss: 2.02653980, train_accuracy: 0.3672, test_Accuracy: 0.3100
Epoch: [ 0] [    9/60000], train_loss: 1.93359292, train_accuracy: 0.3359, test_Accuracy: 0.2817
Epoch: [ 0] [   10/60000], tra

<h1>Sigmoid Function Test Accuracy : 94.04%</h1>

### Lab 10-2: Weight 초기화 잘해보자

Weight 초기화를 잘 해주지 않으면, Locai Minima나 Saddle Point에 빠져서, Global Minima를 찾지 못하게 될 수 있다. 이를 방지하기 위해서, 여러 가지 방법으로 최대한 Global Minima에 가까운 지점으로 Weight를 초기화한다.
<br><br>
1. Xavier Initialization (Glorot Initialization)
<br>: 평균은 0, 분산은 아래와 같은 Random 분포를 사용한다. $${2 \over ChannelIn + ChannelOut}$$<br>
여기서 Channel_In은 input으로 들어가는 채널의 갯수, Channel_Out은 output으로 나오는 채널의 갯수를 의미한다.
<br><br><br>
2. He Initialization
<br>: Relu에 특화된 Weight 초기화. Xavier와 거의 비슷하나, 분산만 2배로 준다는 차이점이 있다. 즉, 평균은 0, 분산은 아래와 같은 Random 분포를 사용한다. $${4 \over ChannelIn + ChannelOut}$$

In [ ]:
# 10-1의 소스 코드와 같다.

import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist # fasion_mnist, cifar10, cifar100과 같은 다양한 데이터셋이 있다.

# mnist 데이터셋을 받아온 뒤 전처리.
def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

  train_data = np.expand_dims(train_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)
  test_data = np.expand_dims(test_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)

  train_data, test_data = normalize(train_data, test_data) # 범위는 0~255 -> 0~1

  train_labels = to_categorical(train_labels, 10)
  test_labels = to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0

  return train_data, test_data

In [ ]:
# 10-1의 소스 코드와 같다.

# shape를 펼쳐주는 함수
# 다차원 배열을 1차원으로 만드는 레이어를 추가한다고 보면 된다.
def flatten():
  return tf.keras.layers.Flatten()

# Dense Layer(Flip Connected Layer) 사용, 케라스 레이어에 Dense 추가
def dense(channel, weight_init):
  return tf.keras.layers.Dense(units=channel, use_bias=True, kernel_initializer=weight_init)

# Relu Activation Function
def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

In [ ]:
# Class type의 모델.
class create_model(tf.keras.Model):
  def __init__(self, label_dim):
    super(create_model, self).__init__()

    # 이 부분을 바꿈으로써 weight 초기화를 할 수 있다.
    # Xavier는 glorot_uniform(), he는 he_uniform().
    weight_init = tf.keras.initializers.he_uniform()
    self.model = tf.keras.Sequential()

    self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]

    for i in range(2):
      # [N, 784] -> [N, 256] -> [N, 256]
      self.model.add(dense(256, weight_init)) # flip-connected function * 2
      self.model.add(relu()) # relu function * 2

    self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
 
  def call(self, x, training=None, mask=None):
    x = self.model(x)

    return x

In [ ]:
# 10-1의 소스 코드와 같다.

# loss 값 구하는 함수.
def loss_fn(model, images, labels):
  logits = model(images, training=True) # images의 숫자를 추출.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)) # softmax로 loss 구함.
  return loss

# 정확도 측정 함수.
# argmax : 가장 큰 숫자값의 위치를 찾는 함수. 여기서는 마지막(-1번째) 차원을 제거한다.
def accuracy_fn(model, images, labels):
  logits = model(images, training=False)
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
  return accuracy

# Gradient Descent 함수.
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)

In [ ]:
# 10-1의 소스 코드와 같다.

""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128 # 이미지를 한 번에 학습시킬 갯수.

training_epochs = 1
training_iterations = len(train_x)

label_dim = 10

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=batch_size).\
  batch(batch_size, drop_remainder=True)
  #repeat() #이를 계속 반복.

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=len(test_x)).\
  batch(len(test_x))
  #repeat()

""" Model """
network = create_model(label_dim)

""" Training """
# 어떤 Optimizer를 써서 loss값을 최소화할 것인가 : 여기서는 AdamOptimizer를 사용.
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# 10-1의 소스 코드와 같다.

# 여기선 체크포인트 구현은 생략했다. (핵심도 아니고, 귀찮고, 코랩 쓰고 있다보니.......)
start_epoch = 0
start_iteration = 0

for epoch in range(start_epoch, training_epochs):
  for idx, (train_input, train_label) in enumerate(train_dataset): #epoch, iteration 2개에 대해 반복 수행.
    # gradient 구한 후 적용, 네트워크를 학습시킨다.
    grads = grad(network, train_input, train_label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

    # loss, 정확도를 구한다.
    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    # test dataset을 불러오고, 정확도를 구한다.
    for test_input, test_label in test_dataset:
      test_accuracy = accuracy_fn(network, test_input, test_label)

    # 그 결과값을 출력한다.
    print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" %(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))

Epoch: [ 0] [    0/60000], train_loss: 2.07590556, train_accuracy: 0.3359, test_Accuracy: 0.1874
Epoch: [ 0] [    1/60000], train_loss: 1.94007158, train_accuracy: 0.3594, test_Accuracy: 0.2799
Epoch: [ 0] [    2/60000], train_loss: 1.96643960, train_accuracy: 0.3438, test_Accuracy: 0.3874
Epoch: [ 0] [    3/60000], train_loss: 1.65629196, train_accuracy: 0.6562, test_Accuracy: 0.5510
Epoch: [ 0] [    4/60000], train_loss: 1.62113547, train_accuracy: 0.6953, test_Accuracy: 0.6776
Epoch: [ 0] [    5/60000], train_loss: 1.54488420, train_accuracy: 0.7031, test_Accuracy: 0.7170
Epoch: [ 0] [    6/60000], train_loss: 1.28875422, train_accuracy: 0.8281, test_Accuracy: 0.7378
Epoch: [ 0] [    7/60000], train_loss: 1.21884727, train_accuracy: 0.7344, test_Accuracy: 0.7592
Epoch: [ 0] [    8/60000], train_loss: 1.03884292, train_accuracy: 0.8438, test_Accuracy: 0.7749
Epoch: [ 0] [    9/60000], train_loss: 1.04049039, train_accuracy: 0.7578, test_Accuracy: 0.7883
Epoch: [ 0] [   10/60000], tra

<h1>Relu Function With He Initialization Test Accuracy : 96.69%</h1>

### Lab 10-3: Dropout

Overfitting을 방지하고, 모델을 잘 학습시키기 위한 Regularization 기법.
- 모델의 노드 중 일부 노드를 끄고 학습시키는 방법. 이 때 끄는 노드는 Random으로 결정된다. 이후 테스트 때는 껏던 노드를 켜서, 모든 노드를 이용한다.
- 랜덤으로 뉴런 일부를 제외함으로써 영향력이 지나치게 강한 뉴런, 즉 학습데이터에 지나치게 의존되어 새로운 데이터에 대한 예측력이 떨어지는 뉴런을 학습에서 랜덤으로 배제할 수 있는 가능성을 얻을 수 있고, 그럴 경우에 과적합을 줄일 수 있다.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

  train_data = np.expand_dims(train_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)
  test_data = np.expand_dims(test_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)

  train_data, test_data = normalize(train_data, test_data) # 범위는 0~255 -> 0~1

  train_labels = to_categorical(train_labels, 10)
  test_labels = to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0

  return train_data, test_data

In [ ]:
# shape를 펼쳐주는 함수
# 다차원 배열을 1차원으로 만드는 레이어를 추가한다고 보면 된다.
def flatten():
  return tf.keras.layers.Flatten()

# Dense Layer(Flip Connected Layer) 사용, 케라스 레이어에 Dense 추가
def dense(channel, weight_init):
  # units = output으로 나가는 채널 갯수, use_bias = bias 사용 여부
  return tf.keras.layers.Dense(units=channel, use_bias=True, kernel_initializer=weight_init)

# Relu Activation Function
def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

# Dropout Function을 추가한다.
# rate : 전체 노드 중 끄는 노드의 비율. 0~1 사이의 값.
def dropout(rate):
  return tf.keras.layers.Dropout(rate)

In [ ]:
# Class type의 모델.
class create_model(tf.keras.Model):
  def __init__(self, label_dim):
    super(create_model, self).__init__()

    # he initialization
    weight_init = tf.keras.initializers.he_uniform()
    self.model = tf.keras.Sequential()

    self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]

    for i in range(2):
      self.model.add(dense(256, weight_init)) # flip-connected function * 2
      self.model.add(relu()) # relu function * 2
      self.model.add(dropout(rate=0.5)) # dropout 추가(절반 정도 끔)

    self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
 
  def call(self, x, training=None, mask=None):
    x = self.model(x)

    return x

In [ ]:
# loss 값 구하는 함수.
def loss_fn(model, images, labels):
  logits = model(images, training=True) # training이 True이면 dropout을 사용한다. (일부 노드로 학습한다.)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)) # softmax로 loss 구함.
  return loss

# 정확도 측정 함수.
def accuracy_fn(model, images, labels):
  logits = model(images, training=False) # training이 False이면 dropout을 사용하지 않는다. (모든 노드를 써서 테스트한다.)
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) # prediction은 Boolean 이기 때문에, 이를 숫자값으로 변경.
  return accuracy

# Gradient Descent 함수.
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)

In [ ]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128 # 이미지를 한 번에 학습시킬 갯수.

training_epochs = 1
training_iterations = len(train_x)

label_dim = 10

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=batch_size).\
  batch(batch_size, drop_remainder=True)
  #repeat() #이를 계속 반복.

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=len(test_x)).\
  batch(len(test_x))
  #repeat()

""" Model """
network = create_model(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [ ]:
""" Model """
network = create_model(label_dim)

# 여기선 체크포인트 구현은 생략했다. (핵심도 아니고, 귀찮고, 코랩 쓰고 있다보니.......)
start_epoch = 0
start_iteration = 0

for epoch in range(start_epoch, training_epochs):
  for idx, (train_input, train_label) in enumerate(train_dataset): #epoch, iteration 2개에 대해 반복 수행.
    # gradient 구한 후 적용, 네트워크를 학습시킨다.
    grads = grad(network, train_input, train_label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

    # loss, 정확도를 구한다.
    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    # test dataset을 불러오고, 정확도를 구한다.
    for test_input, test_label in test_dataset:
      test_accuracy = accuracy_fn(network, test_input, test_label)

    # 그 결과값을 출력한다.
    print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" %(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))

Epoch: [ 0] [    0/60000], train_loss: 2.44036508, train_accuracy: 0.2422, test_Accuracy: 0.1822
Epoch: [ 0] [    1/60000], train_loss: 2.22232723, train_accuracy: 0.3594, test_Accuracy: 0.2955
Epoch: [ 0] [    2/60000], train_loss: 2.28636074, train_accuracy: 0.3594, test_Accuracy: 0.3273
Epoch: [ 0] [    3/60000], train_loss: 2.26121664, train_accuracy: 0.2891, test_Accuracy: 0.3715
Epoch: [ 0] [    4/60000], train_loss: 2.06596875, train_accuracy: 0.4453, test_Accuracy: 0.4100
Epoch: [ 0] [    5/60000], train_loss: 2.13599539, train_accuracy: 0.4531, test_Accuracy: 0.4787
Epoch: [ 0] [    6/60000], train_loss: 2.05927420, train_accuracy: 0.5625, test_Accuracy: 0.5366
Epoch: [ 0] [    7/60000], train_loss: 2.06446075, train_accuracy: 0.5000, test_Accuracy: 0.5737
Epoch: [ 0] [    8/60000], train_loss: 1.96438074, train_accuracy: 0.5938, test_Accuracy: 0.5976
Epoch: [ 0] [    9/60000], train_loss: 1.92078853, train_accuracy: 0.5938, test_Accuracy: 0.6148
Epoch: [ 0] [   10/60000], tra

<h1>Relu Function With He Initialization, Dropout Test Accuracy : 94.98%</h1>

### Lab 10-4: Batch Normalization

데이터의 distribution이 네트워크를 지나가면서 변형이 되는데, 이를 internal covariate shift라고 한다. batch normalization은 이를 막기 위한 방법이다.
<br><br>
레이어의 Input으로 들어오는 distribution을 계속 Normalization함으로써, 이 distribution을 일정하게 유지시킨다. 이후 학습이 되는 파라미터들을 이용해서 새로운 값을 만든 뒤, 이를 레이어의 Input으로 다시 넣어준다.
<br><br><br>
Batch Normalization을 쓰는 경우, Gradient Descent 함수의 수정이 필요하다. <br>기존의 
```python
return tape.gradient(loss, model.variables)
```
를 아래와 같이 바꿔주어야 한다.
```python
return tape.gradient(loss, model.trainable_variables)
```
마찬가지로, Eager mode로 실제 학습을 수행하는 main 함수에서도 아래와 같은 코드 변경이 필요하다.
<br>기존:
```python
optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
```
신규: 
```python
optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))
```

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

  train_data = np.expand_dims(train_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)
  test_data = np.expand_dims(test_data, axis=-1) #(N, 28, 28) -> (N, 28, 28, 1)

  train_data, test_data = normalize(train_data, test_data) # 범위는 0~255 -> 0~1

  train_labels = to_categorical(train_labels, 10)
  test_labels = to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0

  return train_data, test_data

In [ ]:
# shape를 펼쳐주는 함수
# 다차원 배열을 1차원으로 만드는 레이어를 추가한다고 보면 된다.
def flatten():
  return tf.keras.layers.Flatten()

# Dense Layer(Flip Connected Layer) 사용, 케라스 레이어에 Dense 추가
def dense(channel, weight_init):
  # units = output으로 나가는 채널 갯수, use_bias = bias 사용 여부
  return tf.keras.layers.Dense(units=channel, use_bias=True, kernel_initializer=weight_init)

# Relu Activation Function
def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

# 이 예제에서는 dropout이 불필요하므로, batch_normalization으로 변경.
def batch_norm():
  return tf.keras.layers.BatchNormalization()

In [ ]:
# Class type의 모델.
class create_model(tf.keras.Model):
  def __init__(self, label_dim):
    super(create_model, self).__init__()

    # he initialization
    weight_init = tf.keras.initializers.he_uniform()
    self.model = tf.keras.Sequential()

    self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]

    for i in range(2):
      self.model.add(dense(256, weight_init)) # flip-connected function * 2
      self.model.add(batch_norm())
      self.model.add(relu())

    self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
 
  def call(self, x, training=None, mask=None):
    x = self.model(x)

    return x

In [ ]:
# loss 값 구하는 함수.
def loss_fn(model, images, labels):
  logits = model(images, training=True) # training이 True이면 dropout을 사용한다. (일부 노드로 학습한다.)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)) # softmax로 loss 구함.
  return loss

# 정확도 측정 함수.
def accuracy_fn(model, images, labels):
  logits = model(images, training=False) # training이 False이면 dropout을 사용하지 않는다. (모든 노드를 써서 테스트한다.)
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) # prediction은 Boolean 이기 때문에, 이를 숫자값으로 변경.
  return accuracy

# Gradient Descent 함수.
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  #return tape.gradient(loss, model.variables)
  return tape.gradient(loss, model.trainable_variables) #Batch Normalization 추가 시 이 부분의 변경이 필요하다.

In [ ]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128 # 이미지를 한 번에 학습시킬 갯수.

training_epochs = 1
training_iterations = len(train_x)

label_dim = 10

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=batch_size).\
  batch(batch_size, drop_remainder=True)
  #repeat() #이를 계속 반복.

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
  shuffle(buffer_size=100000).\
  prefetch(buffer_size=len(test_x)).\
  batch(len(test_x))
  #repeat()

""" Model """
network = create_model(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [ ]:
""" Model """
network = create_model(label_dim)

# 여기선 체크포인트 구현은 생략했다. (핵심도 아니고, 귀찮고, 코랩 쓰고 있다보니.......)
start_epoch = 0
start_iteration = 0

for epoch in range(start_epoch, training_epochs):
  for idx, (train_input, train_label) in enumerate(train_dataset): #epoch, iteration 2개에 대해 반복 수행.
    # gradient 구한 후 적용, 네트워크를 학습시킨다.
    grads = grad(network, train_input, train_label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

    # loss, 정확도를 구한다.
    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    # test dataset을 불러오고, 정확도를 구한다.
    for test_input, test_label in test_dataset:
      test_accuracy = accuracy_fn(network, test_input, test_label)

    # 그 결과값을 출력한다.
    print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" %(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))

Epoch: [ 0] [    0/60000], train_loss: 1.63932097, train_accuracy: 0.2031, test_Accuracy: 0.2124
Epoch: [ 0] [    1/60000], train_loss: 1.41592872, train_accuracy: 0.3516, test_Accuracy: 0.3225
Epoch: [ 0] [    2/60000], train_loss: 1.24864984, train_accuracy: 0.5312, test_Accuracy: 0.4074
Epoch: [ 0] [    3/60000], train_loss: 1.14556110, train_accuracy: 0.5156, test_Accuracy: 0.4779
Epoch: [ 0] [    4/60000], train_loss: 1.00390577, train_accuracy: 0.5312, test_Accuracy: 0.5273
Epoch: [ 0] [    5/60000], train_loss: 0.81152678, train_accuracy: 0.5469, test_Accuracy: 0.5713
Epoch: [ 0] [    6/60000], train_loss: 0.72763312, train_accuracy: 0.6875, test_Accuracy: 0.5980
Epoch: [ 0] [    7/60000], train_loss: 0.69771606, train_accuracy: 0.6250, test_Accuracy: 0.6238
Epoch: [ 0] [    8/60000], train_loss: 0.71162271, train_accuracy: 0.6797, test_Accuracy: 0.6453
Epoch: [ 0] [    9/60000], train_loss: 0.62007040, train_accuracy: 0.6250, test_Accuracy: 0.6592
Epoch: [ 0] [   10/60000], tra

<h1>Relu Function With He Initialization, Batch Normalization Test Accuracy : 97.02%</h1>